In [1]:
import pandas as pd
import datetime
from bs4 import BeautifulSoup
import requests

## Import historical game by game data from FiveThirtyEight GitHub Repo

[FiveThirtyEight Data Repo](https://github.com/fivethirtyeight/data)

In [189]:
url = "https://projects.fivethirtyeight.com/nba-model/nba_elo.csv"
df = pd.read_csv(url).astype({'date': 'datetime64[ns]'})
df.head()

,date,season,neutral,playoff,team1,team2,elo1_pre,elo2_pre,elo_prob1,elo_prob2,elo1_post,elo2_post,carmelo1_pre,carmelo2_pre,carmelo1_post,carmelo2_post,carmelo_prob1,carmelo_prob2,score1,score2
0,1946-11-01,1947,0,NaN,TRH,NYK,1300.0,1300.0000,0.640065,0.359935,1293.2767,1306.7233,NaN,NaN,NaN,NaN,NaN,NaN,66.0,68.0
1,1946-11-02,1947,0,NaN,CHS,NYK,1300.0,1306.7233,0.631101,0.368899,1309.6521,1297.0712,NaN,NaN,NaN,NaN,NaN,NaN,63.0,47.0
2,1946-11-02,1947,0,NaN,PRO,BOS,1300.0,1300.0000,0.640065,0.359935,1305.1542,1294.8458,NaN,NaN,NaN,NaN,NaN,NaN,59.0,53.0
3,1946-11-02,1947,0,NaN,STB,PIT,1300.0,1300.0000,0.640065,0.359935,1304.6908,1295.3092,NaN,NaN,NaN,NaN,NaN,NaN,56.0,51.0
4,1946-11-02,1947,0,NaN,DTF,WSC,1300.0,1300.0000,0.640065,0.359935,1279.6189,1320.3811,NaN,NaN,NaN,NaN,NaN,NaN,33.0,50.0


### Analyze fields

See [FiveThirtyEight NBA Predictions](https://projects.fivethirtyeight.com/2018-nba-predictions/) for explanation of Elo and CARM-Elo

| Name           |  Type         | Description                                                                        
| -------------  |-------------  | ----------------------------------------------------------------------------------|
| date           | datetime      |   Game date                                                                       |                      
| season         | int           |   End year of season                                                              |
| neutral        | bool          |   1=neutral home court                                                            |
| playoff        | string        |   1946-2015: t = playoffs. Starting in 2016: q=conference quarterfinals, s=conference semifinals, c=conference final,f=finals                                                                 |
| team1          | string        |   Home team initials                                                              |
| team2          | string        |   Away team initials                                                              |
| elo1_pre       | double        |   Home team Elo before game                                                       |
| elo2_pre       | double        |   Away team Elo before game                                                       |
| elo_prob1      | double        |   Win probability based on elo1_pre                                               |
| elo_prob2      | double        |   Win probability based on elo2_pre                                               |
| elo1_post      | double        |   Home team Elo after game                                                        |
| elo2_post      | double        |   Away team Elo after game                                                        |
| carmelo1_pre\* | double        |   Home team CARM-Elo before game                                                  |
| carmelo2_pre\* | double        |   Away team CARM-Elo before game                                                  |
| carmelo1_post\*| double        |   Home team CARM-Elo after game                                                   |
| carmelo2_post\*| double        |   Away team CARM-Elo after game                                                   |
| carmelo_prob1\*| double        |   Win probability based on carmelo1_pre                                           |
| carmelo_prob2\*| double        |   Win probability based on carmelo2_pre                                           |
| score1         | int           |   Home team final score                                                           |
| score2         | int           |   Away team final score                                                           |

\* data only available from 2015-2018

#### Determine CARM-Elo starting point

In [ ]:
df_carm_elo = df[["date", "season", "team1", "team2", "carmelo1_pre", "carmelo2_pre", "carmelo1_post", "carmelo2_post"]]
df_carm_elo[~(pd.isnull(df_carm_elo["carmelo1_pre"]))].sort_values("date").head()

#### CARM-Elo data available starting with 2015-2016 season

## Import historical season data from basketball-reference

[Basketball-Reference Miscellaneous season stats](https://www.basketball-reference.com/leagues/NBA_2018.html#misc_stats::none)

In [3]:
def get_season_data(end_year):
    ''' 
    get cumulative statistics for season specified by end_year
    
    end_year: int, year to query (ex: 2018 queries 2017-2018 season)
    
    returns Pandas dataframe w/ basketball-reference.com's miscellaneous stats table for season specified by end_year
    '''
    from bs4 import Comment
    html = "https://www.basketball-reference.com/leagues/NBA_{}.html".format(end_year)
    result = requests.get(html)
    soup = BeautifulSoup(result.content, "html.parser")
    # html tree is strange...table is wrapped inside a comment
    table = [c for c in (soup.find('div', id="all_misc_stats")).children if type(c) == Comment][0]
    # parse table with pandas
    df = pd.read_html(table, header=1)[0]
    df["Season"] = end_year
    return df

#### Option 1) Download latest data

In [ ]:
start_year = 1951
end_year = 2018
dfs_1951_2018 = [get_season_data(i) for i in range(1951, 2019)]
df_1951_2018 = pd.concat(dfs_1951_2018)
df_1951_2018["Team"] =  df_1951_2018["Team"].map(lambda s: s.replace("*", ""))
df_1951_2018 = df_1951_2018.set_index(["Season", "Team"])
pd.set_option('display.max_columns', 30)
df_1951_2018.head()
# Save
# df_1951_2018.to_csv("../Data/nba_season_data.csv")

#### Option 2) Use saved offline file

In [4]:
df_1951_2018 = pd.read_csv("../Data/nba_season_data.csv")
df_1951_2018["Team"] =  df_1951_2018["Team"].map(lambda s: s.replace("*", ""))
df_1951_2018 = df_1951_2018.set_index(["Season", "Team"])
pd.set_option('display.max_columns', 30)
df_1951_2018.head()

Rk  Age     W     L    PW    PL   MOV   SOS  \
Season Team                                                                  
1951   Minneapolis Lakers     1.0  NaN  44.0  24.0  49.0  19.0  5.41 -0.63   
       Philadelphia Warriors  2.0  NaN  40.0  26.0  43.0  23.0  3.76 -0.36   
       Rochester Royals       3.0  NaN  41.0  27.0  42.0  26.0  2.99 -0.44   
       Syracuse Nationals     4.0  NaN  32.0  34.0  34.0  32.0  0.53  0.09   
       New York Knicks        5.0  NaN  36.0  30.0  34.0  32.0  0.41  0.07   

                               SRS  ORtg  DRtg  Pace    FTr  3PAr    TS%  \
Season Team                                                                
1951   Minneapolis Lakers     4.79  86.4  80.7  94.8  0.356   NaN  0.436   
       Philadelphia Warriors  3.40  84.8  81.0  99.3  0.385   NaN  0.425   
       Rochester Royals       2.54  89.3  86.2  92.3  0.418   NaN  0.452   
       Syracuse Nationals     0.62  86.4  85.9  98.1  0.491   NaN  0.435   
       New York Knicks        0.49  88.0  87.6  94.8  0.415   NaN  0.445   

                               eFG%  TOV%  ORB%  FT/FGA  eFG%.1  TOV%.1  DRB%  \
Season Team                                                                     
1951   Minneapolis Lakers     0.373   NaN   NaN   0.262     NaN     NaN   NaN   
       Philadelphia Warriors  0.350   NaN   NaN   0.294     NaN     NaN   NaN   
       Rochester Royals       0.378   NaN   NaN   0.315     NaN     NaN   NaN   
       Syracuse Nationals     0.351   NaN   NaN   0.356     NaN     NaN   NaN   
       New York Knicks        0.379   NaN   NaN   0.296     NaN     NaN   NaN   

                              FT/FGA.1                        Arena  Attend.  \
Season Team                                                                    
1951   Minneapolis Lakers          NaN       Minneapolis Auditorium      NaN   
       Philadelphia Warriors       NaN           Philadelphia Arena      NaN   
       Rochester Royals            NaN          Edgerton Park Arena      NaN   
       Syracuse Nationals          NaN          State Fair Coliseum      NaN   
       New York Knicks             NaN  Madison Square Garden (III)      NaN   

                              Attend./G  
Season Team                              
1951   Minneapolis Lakers           NaN  
       Philadelphia Warriors        NaN  
       Rochester Royals             NaN  
       Syracuse Nationals           NaN  
       New York Knicks              NaN

### Analyze fields

| Name      |  Type  | Description                                                                                   |
| --------- |--------| --------------------------------------------------------------------------------------------- |
| Rk        | double |   Rank (used to index for sorting within webpage)                                             |
| Age       | double |   Age of Player at the start of February 1st of that season.                                  |
| W         | double |   Wins                                                                                        |
| L         | double |   Losses                                                                                      |
| PW        | double |   Pythagorean wins, i.e., expected wins based on points scored and allowed                    |
| PL        | double |   Pythagorean losses, i.e., expected losses based on points scored and allowed                |
| MOV       | double |   Margin of Victory                                                                           |
| SOS       | double |   Strength of Schedule; a rating of strength of schedule. The rating is denominated in points above/below average, where zero is average.                                                                          |
| SRS       | double |   Simple Rating System; a team rating that takes into account average point differential and strength of schedule. The rating is denominated in points above/below average, where zero is average.                |
| ORtg      | double |   An estimate of points produced (players) or scored (teams) per 100 possessions              |  
| DRtg      | double |   An estimate of points allowed per 100 possessions                                           |
| Pace      | double |   An estimate of possessions per 48 minutes                                                   |
| FTr       | double |   Number of FT Attempts Per FG Attempt                                                        |
| 3PAr\*\*  | double |   Percentage of FG Attempts from 3-Point Range                                                |
| TS%       | double |   A measure of shooting efficiency that takes into account 2-point field goals, 3-point field goals, and free throws.                                                                                              |
| eFG%      | double |   Adjusts for the fact that a 3-point field goal is worth one more point than a 2-point field goal.                                                                                                                |
| TOV%\*    | double |   An estimate of turnovers committed per 100 plays.                                           |
| ORB%\*    | int    |   An estimate of the percentage of available offensive rebounds a team grabbed.               |
| FT/FGA    | int    |   Free Throws Per Field Goal Attempt                                                          |
| eFG%.1    | double |    Opponent Effective Field Goal Percentage                                                   |
| TOV%.1\*  | double |   Opponent Turnover Percentage                                                                |
| DRB%\*    | int    |   An estimate of the percentage of available defensive rebounds a team grabbed.               |
| FT/FGA.1\*| int    |   Opponent Free Throws Per Field Goal Attempt                                                 |    
| Arena     | string |   Home Arena                                                                                  |
| Attend.   | double |   Cumulative home attendance                                                                  |
| Attend./G | double |   Attendance per home game at the team's primary arena                                        |

\* data only available from 1974-2018

\*\* data only available from 1980-2018

NOTE: No data available for 1954-1955 Baltimore Bullets

## Import additional historical game by game data from basketball-reference

Starting from the 1983-1984 season, basketball-reference.com has game by game advanced statistics such as ORtg, DRtg, and eFG%

[Example boxscore](https://www.basketball-reference.com/boxscores/201803050CHI.html)

In [112]:
def boxscore_links_for_date(date):
    '''
    get list of basketball-reference links to boxscores for games on given date
    
    date: datetime.datetime object with year, month, and day specified
    
    returns list of urls to basketball-reference single game boxscores for given date
    '''
    link = "https://www.basketball-reference.com/boxscores/?month={}&day={}&year={}".format(date.month, date.day, date.year)
    result = requests.get(link)
    soup = BeautifulSoup(result.content, "html.parser")
    return ["http://www.basketball-reference.com" + game.find("a").get("href") for game in soup.find_all("td", {"class": "right gamelink"})]

In [109]:
def boxscore_dict_for_link(link):
    '''
    advanced box score stats as dictionary from basketball-reference boxscore link
    
    link: string, link to single game basketball-reference boxscore
    
    returns dictionary with advanced stats for home (team1) and away (team2) teams for boxscore linked
    NOTE: basketball-reference only supports single game advanced stats starting from 1983-1984 NBA season
    '''
    result = requests.get(link)
    soup = BeautifulSoup(result.content, "html.parser")
    from bs4 import Comment
    assert soup.find("div", id="all_four_factors") != None, "Advanced box score metrics only available for dates with at least 1 NBA game starting from 1983-1984 season"
    for c in soup.find("div", id="all_four_factors").children:
        if type(c) == Comment:
            s_ind = c.index("<table")
            e_ind = c.index("</table>")
            table_html = c[s_ind:e_ind+8]
            break
    df = pd.read_html(table_html, header=1, index_col=0)[0]
    df["DRtg"] = df["ORtg"].values[::-1]
    df["NetRtg"] = df["ORtg"] - df["DRtg"]
    d = {}
    for i, team_name in enumerate(df.index):
        prefix = "team2_" if i == 0 else "team1_"
        for col in df.columns:
            d[prefix+col] = df.loc[team_name, col]
    return d

#### Crawling through nba_elo data to find advanced box score metrics for all games since 1983-1984 NBA season

In [207]:
# Basketball-reference.com has advanced metrics for single games starting w/ 1983-1984 NBA season
season_start_dates = []
season_end_dates = []
for i in range(1985, 2019):
    season_df = df[(df["season"] == i) & (~pd.isnull(df["score1"]))]
    season_start_dates.append(season_df.head(1)["date"])
    season_end_dates.append(season_df.tail(1)["date"])
# Query basketball reference game by game, and save results for each year
for season_start_date, season_end_date in zip(season_start_dates, season_end_dates):
    df_slice = df[(df["date"] >= season_start_date) & (df["date"] <= season_end_date)].copy()
    current_date = season_start_date
    boxscores_for_date = []
    boxscores = boxscore_links_for_date(current_date)
    data = []
    for _, row in df_slice.iterrows():
        print("{} vs. {} on {}".format(row["team1"], row["team2"], row["date"]))
        if current_date != row["date"]:
            # get new boxscores for date
            print("New day ({}), getting boxscores".format(row["date"]))
            boxscores = boxscore_links_for_date(row["date"])
            current_date = row["date"]
        home_team = row["team1"]
        boxscore_for_game = list(filter(lambda link: home_team in link, boxscores))[0]
        d = boxscore_dict_for_link(boxscore_for_game)
        for key, val in d.items():
            row[key] = val
        data.append(row)
    # save data for year
    print("Saving for season from {} to {}".format(season_start_date, season_end_date))
    pd.DataFrame(data).to_csv("nba_game_data_{}-{}-{}_to_{}-{}-{}.csv".format(season_start_date.year, season_start_date.month, season_start_date.day, season_end_date.year, season_end_date.month, season_end_date.day))

In [184]:
# Basketball-reference.com has advanced metrics for single games starting w/ 1983-1984 NBA season
# Query basketball reference game by game, and save results for each year
start_date = datetime.datetime(1984, 1, 27)
end_date = datetime.datetime(1984, 6, 12)
df_slice = df[(df["date"] >= start_date) & (df["date"] <= end_date)].copy()
current_date = start_date
fn_start_date = start_date
boxscores_for_date = []
boxscores = boxscore_links_for_date(current_date)
data = []
for i, row in df_slice.iterrows():
    print("{} vs. {} on {}".format(row["team1"], row["team2"], row["date"]))
    if current_date != row["date"]:
#         if row["date"].month != current_date.month:
#             print("New month ({}), saving results w/ fn_start_date = {}, current_date = {}".format(row["date"].month, fn_start_date, current_date))
#             # save results
#             pd.DataFrame(data).to_csv("nba_game_data_{}-{}-{}_to_{}-{}-{}.csv".format(fn_start_date.year, fn_start_date.month, fn_start_date.day, current_date.year, current_date.month, current_date.day))
#             fn_start_date = row["date"]
#             data = []
        # get new boxscores for date
        print("New day ({}), getting boxscores".format(row["date"]))
        boxscores = boxscore_links_for_date(row["date"])
        current_date = row["date"]
    home_team = row["team1"]
    boxscore_for_game = list(filter(lambda link: home_team in link, boxscores))[0]
    d = boxscore_dict_for_link(boxscore_for_game)
    for key, val in d.items():
        row[key] = val
    data.append(row)
# final save
pd.DataFrame(data).to_csv("nba_game_data_{}-{}-{}_to_{}-{}-{}.csv".format(start_date.year, start_date.month, start_date.day, current_date.year, current_date.month, current_date.day))

SAS vs. NYK on 1984-01-31 00:00:00
New day (1984-01-31 00:00:00), getting boxscores
CHI vs. BOS on 1984-01-31 00:00:00
LAL vs. POR on 1984-01-31 00:00:00
KCK vs. HOU on 1984-01-31 00:00:00
CLE vs. DET on 1984-01-31 00:00:00
IND vs. SDC on 1984-01-31 00:00:00
GSW vs. MIL on 1984-01-31 00:00:00
WSB vs. ATL on 1984-01-31 00:00:00
UTA vs. SEA on 1984-01-31 00:00:00
NJN vs. SDC on 1984-02-01 00:00:00
New day (1984-02-01 00:00:00), getting boxscores
DEN vs. LAL on 1984-02-01 00:00:00
ATL vs. CHI on 1984-02-01 00:00:00
DAL vs. NYK on 1984-02-01 00:00:00
BOS vs. KCK on 1984-02-01 00:00:00
SEA vs. MIL on 1984-02-01 00:00:00
PHO vs. POR on 1984-02-01 00:00:00
PHI vs. IND on 1984-02-01 00:00:00
WSB vs. DET on 1984-02-02 00:00:00
New day (1984-02-02 00:00:00), getting boxscores
GSW vs. SAS on 1984-02-02 00:00:00
UTA vs. PHO on 1984-02-02 00:00:00
HOU vs. DAL on 1984-02-02 00:00:00
DAL vs. DEN on 1984-02-03 00:00:00
New day (1984-02-03 00:00:00), getting boxscores
CLE vs. CHI on 1984-02-03 00:00:00

In [185]:
fns = [
    '../Data/nba_game_data_1983-10-28_to_1983-10-30.csv',
    '../Data/nba_game_data_1983-11-1_to_1983-11-30.csv',
    '../Data/nba_game_data_1983-12-1_to_1983-12-30.csv',
    '../Data/nba_game_data_1984-1-1_to_1984-1-26.csv',
    '../Data/nba_game_data_1984-1-27_to_1984-6-12.csv'
]
df = pd.concat([pd.read_csv(fn, index_col=0) for fn in fns])
df

,date,season,neutral,playoff,team1,team2,elo1_pre,elo2_pre,elo_prob1,elo_prob2,elo1_post,elo2_post,carmelo1_pre,carmelo2_pre,carmelo1_post,...,team2_eFG%,team2_TOV%,team2_ORB%,team2_FT/FGA,team2_ORtg,team2_DRtg,team2_NetRtg,team1_Pace,team1_eFG%,team1_TOV%,team1_ORB%,team1_FT/FGA,team1_ORtg,team1_DRtg,team1_NetRtg
24793,1983-10-28,1984,0,NaN,CLE,NYK,1381.7836,1585.6497,0.354824,0.645176,1376.2715,1591.1617,NaN,NaN,NaN,...,0.590,15.6,48.5,0.269,124.5,116.8,7.7,90.7,0.513,15.8,48.6,0.333,116.8,124.5,-7.7
24794,1983-10-28,1984,0,NaN,PHI,WSB,1680.4036,1542.6188,0.797186,0.202814,1682.3088,1540.7135,NaN,NaN,NaN,...,0.561,18.1,27.8,0.268,110.1,113.0,-2.9,103.5,0.512,16.6,36.6,0.402,113.0,110.1,2.9
24795,1983-10-28,1984,0,NaN,DEN,UTA,1538.7487,1430.0630,0.768754,0.231246,1543.8461,1424.9656,NaN,NaN,NaN,...,0.500,5.8,21.3,0.250,115.6,128.6,-13.0,108.1,0.539,8.5,41.3,0.284,128.6,115.6,13.0
24796,1983-10-28,1984,0,NaN,POR,SDC,1529.3440,1378.8689,0.808741,0.191259,1533.4425,1374.7704,NaN,NaN,NaN,...,0.500,17.8,20.6,0.348,97.9,112.6,-14.7,95.0,0.506,15.3,27.3,0.309,112.6,97.9,14.7
24797,1983-10-28,1984,0,NaN,SEA,GSW,1540.0374,1436.8497,0.763080,0.236920,1532.6714,1444.2157,NaN,NaN,NaN,...,0.472,15.3,47.7,0.292,108.3,107.3,1.0,101.6,0.534,19.1,23.5,0.405,107.3,108.3,-1.0
24798,1983-10-28,1984,0,NaN,DET,BOS,1481.0853,1545.0776,0.551635,0.448365,1487.8253,1538.3376,NaN,NaN,NaN,...,0.471,17.3,30.8,0.465,108.7,114.1,-5.4,111.3,0.420,8.9,32.7,0.430,114.1,108.7,5.4
24799,1983-10-28,1984,0,NaN,NJN,ATL,1544.6565,1502.9988,0.693269,0.306731,1553.0487,1494.6066,NaN,NaN,NaN,...,0.456,13.7,32.6,0.289,103.3,120.5,-17.2,104.6,0.574,15.9,50.0,0.284,120.5,103.3,17.2
24800,1983-10-28,1984,0,NaN,KCK,LAL,1531.5054,1581.1077,0.572024,0.427976,1519.1343,1593.4788,NaN,NaN,NaN,...,0.551,14.5,27.3,0.213,113.1,103.4,9.7,103.5,0.484,17.6,42.0,0.183,103.4,113.1,-9.7
24801,1983-10-28,1984,0,NaN,MIL,IND,1577.8187,1367.3145,0.856609,0.143391,1581.7119,1363.4213,NaN,NaN,NaN,...,0.337,14.4,38.1,0.125,81.0,101.5,-20.5,102.5,0.433,14.9,42.0,0.206,101.5,81.0,20.5
24802,1983-10-29,1984,0,NaN,IND,PHI,1363.4213,1682.3088,0.220971,0.779029,1359.0450,1686.6851,NaN,NaN,NaN,...,0.536,20.7,40.9,0.422,111.7,100.9,10.8,100.6,0.453,19.4,39.2,0.274,100.9,111.7,-10.8


In [187]:
df.to_csv("../Data/nba_game_data_1984.csv")